# 0. Prep enviornment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: get github repository https://github.com/BayesRulez/snomed_el_baseline_model.git

!git clone https://github.com/BayesRulez/snomed_el_baseline_model.git
!cp /content/snomed_el_baseline_model/snomed_graph.py /content/


Cloning into 'snomed_el_baseline_model'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 6), reused 11 (delta 2), pack-reused 0
Receiving objects: 100% (18/18), 33.47 KiB | 33.47 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
# delete
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Change directory to SNOMED_CT folder
%cd /content/drive/My\ Drive/SNOMED_CT


Mounted at /content/drive
/content/drive/My Drive/SNOMED_CT


In [ ]:
# Get the data from the specified folder in your Google Drive
!cp -a /content/drive/My\ Drive/SNOMED_CT/* /content/


In [ ]:
!pip install tiktoken
!pip install pickle
!pip install dill
!pip install evaluate
!pip install ipymarkup
!pip install peft
!pip install transformers
!pip install sentence_transformers
!pip install torch
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Un

In [ ]:
from itertools import combinations

import dill as pickle
import evaluate
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from datasets import Dataset
from gensim.models.keyedvectors import KeyedVectors
from ipymarkup import show_span_line_markup
from more_itertools import chunked
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from sentence_transformers import InputExample, SentenceTransformer, losses, models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    DebertaV2ForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)
from snomed_graph import *


In [ ]:
random_seed = 42  # For reproducibility
max_seq_len = 512  # Maximum sequence length for (BERT-based) encoders
cer_model_id = "microsoft/deberta-v3-large"  # Base model for Clinical Entity Recogniser
kb_embedding_model_id = ("sentence-transformers/all-MiniLM-L6-v2") # base model for concept encoder
use_LoRA = True  # Whether to use a LoRA to fine-tune the CER model


In [ ]:
import torch
torch.manual_seed(random_seed)
assert torch.cuda.is_available()


In [ ]:
from snomed_graph import SnomedGraph
SG = SnomedGraph.from_rf2("SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z")
# SG = SnomedGraph.from_rf2("SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z_Challenge_Edition")


FileNotFoundError: [Errno 2] No such file or directory: 'SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z/Snapshot/Terminology/sct2_Relationship_Snapshot_INT_20230531.txt'

# Load Data

In [ ]:
# import pandas as pd
notes_df = pd.read_csv("/content/Notes2SNOMED/data/mimic-iv_notes_training_set.csv").set_index("note_id")
print(f"{notes_df.shape[0]} notes loaded.")

204 notes loaded.


In [ ]:
annotations_df = pd.read_csv("/content/Notes2SNOMED/data/train_annotations.csv").set_index("note_id")
print(f"{annotations_df.shape[0]} annotations loaded.")
print(f"{annotations_df.concept_id.nunique()} unique concepts seen.")
print(f"{annotations_df.index.nunique()} unique notes seen.")


In [ ]:
training_notes_df, test_notes_df = train_test_split(
    notes_df, test_size=32, random_state=random_seed
)
training_annotations_df = annotations_df.loc[training_notes_df.index]
test_annotations_df = annotations_df.loc[test_notes_df.index]

print(
    f"There are {training_annotations_df.shape[0]} total annotations in the training set."
)
print(f"There are {test_annotations_df.shape[0]} total annotations in the test set.")
print(
    f"There are {training_annotations_df.concept_id.nunique()} distinct concepts in the training set."
)
print(
    f"There are {test_annotations_df.concept_id.nunique()} distinct concepts in the test set."
)
print(f"There are {training_notes_df.shape[0]} notes in the training set.")
print(f"There are {test_notes_df.shape[0]} notes in the test set.")


There are 44075 total annotations in the training set.
There are 7499 total annotations in the test set.
There are 4924 distinct concepts in the training set.
There are 1799 distinct concepts in the test set.
There are 172 notes in the training set.
There are 32 notes in the test set.


# Train the CER model

In [ ]:
label2id = {"O": 0, "B-clinical_entity": 1, "I-clinical_entity": 2}

id2label = {v: k for k, v in label2id.items()}
cer_tokenizer = AutoTokenizer.from_pretrained(
    cer_model_id, model_max_length=max_seq_len
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def get_annotation_boundaries(note_id, annotations_df):
    for row in annotations_df.loc[note_id].itertuples():
        yield row.start, row.end, row.concept_id
    yield 1000000, 1000000, None


In [ ]:
def generate_ner_dataset(notes_df, annotations_df):
    for row in notes_df.itertuples():
        tokenized = cer_tokenizer(
            row.text,
            return_offsets_mapping=False,  # Avoid misalignments due to destructive tokenization
            return_token_type_ids=False,  # We're going to construct these below
            return_attention_mask=False,  # We'll construct this by hand
            add_special_tokens=False,  # We'll add these by hand
            truncation=False,  # We'll chunk the notes ourselves
        )

        # Prime the annotation generator and fetch the token <-> word_id map
        annotation_boundaries = get_annotation_boundaries(row.Index, annotations_df)
        ann_start, ann_end, concept_id = next(annotation_boundaries)
        word_ids = tokenized.word_ids()

        # The offsets_mapping returned by the tokenizer will be misaligned vs the original text.
        # This is due to the fact that the tokenization scheme is destructive, for example it
        # drops spaces which cannot be recovered when decoding the inputs.
        # In the following code snippet we create an offset mapping which is aligned with the
        # original text; hence we can accurately locate the annotations and match them to the
        # tokens.
        global_offset = 0
        global_offset_mapping = []

        for input_id in tokenized["input_ids"]:
            token = cer_tokenizer.decode(input_id)
            pos = row.text[global_offset:].find(token)
            start = global_offset + pos
            end = global_offset + pos + len(token)
            global_offset = end
            global_offset_mapping.append((start, end))

        # Note the max_seq_len - 2.
        # This is because we will have to add [CLS] and [SEP] tokens once we're done.
        it = zip(
            chunked(tokenized["input_ids"], max_seq_len - 2),
            chunked(global_offset_mapping, max_seq_len - 2),
            chunked(word_ids, max_seq_len - 2),
        )

        # Since we are chunking the discharge notes, we need to maintain the start and
        # end character index for each chunk so that we can align the annotations for
        # chunks > 1
        chunk_start_idx = 0
        chunk_end_idx = 0

        for chunk_id, chunk in enumerate(it):
            input_id_chunk, offset_mapping_chunk, word_id_chunk = chunk
            token_type_chunk = list()
            concept_id_chunk = list()
            prev_word_id = -1
            concept_word_number = 0
            chunk_start_idx = chunk_end_idx
            chunk_end_idx = offset_mapping_chunk[-1][1]

            for offsets, word_id in zip(offset_mapping_chunk, word_id_chunk):
                token_start, token_end = offsets

                # Check whether we need to fetch the next annotation
                if token_start >= ann_end:
                    ann_start, ann_end, concept_id = next(annotation_boundaries)
                    concept_word_number = 0

                # Check whether the token's position overlaps with the next annotation
                if token_start < ann_end and token_end > ann_start:
                    if prev_word_id != word_id:
                        concept_word_number += 1

                    # If so, annotate based on the word number in the concept
                    if concept_word_number == 1:
                        token_type_chunk.append(label2id["B-clinical_entity"])
                    else:
                        token_type_chunk.append(label2id["I-clinical_entity"])

                    # Add the SCTID (we'll use this later to train the Linker)
                    concept_id_chunk.append(concept_id)

                # Not part of an annotation
                else:
                    token_type_chunk.append(label2id["O"])
                    concept_id_chunk.append(None)

                prev_word_id = word_id

            # Manually adding the [CLS] and [SEP] tokens.
            token_type_chunk = [-100] + token_type_chunk + [-100]
            input_id_chunk = (
                [cer_tokenizer.cls_token_id]
                + input_id_chunk
                + [cer_tokenizer.sep_token_id]
            )
            attention_mask_chunk = [1] * len(input_id_chunk)
            offset_mapping_chunk = (
                [(None, None)] + offset_mapping_chunk + [(None, None)]
            )
            concept_id_chunk = [None] + concept_id_chunk + [None]

            yield {
                # These are the fields we need
                "note_id": row.Index,
                "input_ids": input_id_chunk,
                "attention_mask": attention_mask_chunk,
                "labels": token_type_chunk,
                # These fields are helpful for debugging
                "chunk_id": chunk_id,
                "chunk_span": (chunk_start_idx, chunk_end_idx),
                "offset_mapping": offset_mapping_chunk,
                "text": row.text[chunk_start_idx:chunk_end_idx],
                "concept_ids": concept_id_chunk,
            }


In [ ]:
train = pd.DataFrame(
    list(generate_ner_dataset(training_notes_df, training_annotations_df))
)
train = Dataset.from_pandas(train)
train

Token indices sequence length is longer than the specified maximum sequence length for this model (1629 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['note_id', 'input_ids', 'attention_mask', 'labels', 'chunk_id', 'chunk_span', 'offset_mapping', 'text', 'concept_ids'],
    num_rows: 920
})

In [ ]:
test = pd.DataFrame(list(generate_ner_dataset(test_notes_df, test_annotations_df)))
test = Dataset.from_pandas(test)
test

Dataset({
    features: ['note_id', 'input_ids', 'attention_mask', 'labels', 'chunk_id', 'chunk_span', 'offset_mapping', 'text', 'concept_ids'],
    num_rows: 152
})

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=cer_tokenizer)


### define some training metrics

In [ ]:
seqeval = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
cer_model = DebertaV2ForTokenClassification.from_pretrained(
    cer_model_id, num_labels=3, id2label=id2label, label2id=label2id
)

if use_LoRA:
    lora_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="TOKEN_CLS",
    )

    cer_model = get_peft_model(cer_model, lora_config)

    cer_model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 789,507 || all params: 434,804,742 || trainable%: 0.18157736651363385


In [ ]:
training_args = TrainingArguments(
    output_dir="~/Ntes2SNOMED/cer_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=False,
    seed=random_seed,
)

trainer = Trainer(
    model=cer_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=cer_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.637600,0.633226,0.121212,0.000378,0.000754,0.794741
2,0.499200,0.482094,0.577999,0.115217,0.192135,0.809231
3,0.363800,0.370168,0.530324,0.514083,0.522077,0.855640


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.637600,0.633226,0.121212,0.000378,0.000754,0.794741
2,0.499200,0.482094,0.577999,0.115217,0.192135,0.809231
3,0.363800,0.370168,0.530324,0.514083,0.522077,0.855640
4,0.344100,0.332008,0.516746,0.618336,0.562995,0.870696
5,0.330200,0.314126,0.544818,0.639414,0.588338,0.879206
6,0.320200,0.305789,0.552546,0.672873,0.606802,0.883757
7,0.344200,0.299251,0.571222,0.671267,0.617216,0.887376
8,0.301700,0.296825,0.568873,0.691304,0.624141,0.887716
9,0.319200,0.289015,0.585494,0.682892,0.630454,0.890910
10,0.291300,0.285168,0.591278,0.689414,0.636586,0.892621


TrainOutput(global_step=2300, training_loss=0.33930091111556343, metrics={'train_runtime': 1605.4821, 'train_samples_per_second': 11.461, 'train_steps_per_second': 1.433, 'total_flos': 1.71330090811392e+16, 'train_loss': 0.33930091111556343, 'epoch': 20.0})

In [ ]:
trainer.save_model("./content/Notes2SNOMED/cer_model")
cer_tokenizer.save_pretrained("./content/Notes2SNOMED/cer_model")


('./content/Notes2SNOMED/cer_model/tokenizer_config.json',
 './content/Notes2SNOMED/cer_model/special_tokens_map.json',
 './content/Notes2SNOMED/cer_model/spm.model',
 './content/Notes2SNOMED/cer_model/added_tokens.json',
 './content/Notes2SNOMED/cer_model/tokenizer.json')

In [ ]:
if use_LoRA:
    config = PeftConfig.from_pretrained("./Notes2SNOMED/cer_model")

    cer_model = DebertaV2ForTokenClassification.from_pretrained(
        pretrained_model_name_or_path=config.base_model_name_or_path,
        num_labels=3,
        id2label=id2label,
        label2id=label2id,
    )
    cer_model = PeftModel.from_pretrained(cer_model, "./Notes2SNOMED/cer_model")
else:
    cer_model = DebertaV2ForTokenClassification.from_pretrained(
        pretrained_model_name_or_path="./Notes2SNOMED/cer_model",
        num_labels=3,
        id2label=id2label,
        label2id=label2id,
    )

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# If using the adaptor, ignore the warning:
# "The model 'PeftModelForTokenClassification' is not supported for token-classification."
# The PEFT model is wrapped just fine and will work within the pipeline.
# N.B. moving model to CPU makes inference slower, but enables us to feed the pipeline
# directly with strings.
cer_pipeline = pipeline(
    task="token-classification",
    model=cer_model,
    tokenizer=cer_tokenizer,
    aggregation_strategy="first",
    device="cuda", # cpu
)


The model 'PeftModelForTokenClassification' is not supported for token-classification. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GPT2ForTokenClassification', 'GPT2ForTokenClassification', 'GPTBigCodeForTokenClassification', 'GPTNeoForTokenClassification', 'GPTNeoXForTokenClassification', 'IBertForTokenClassification', 'LayoutLMForTok

In [ ]:

note_id = "10807423-DS-19"
text = test_notes_df.loc[note_id].text

# +1 to offset the [CLS] token which will have been added by the tokenizer
predicted_annotations = [
    (span["start"] + 1, span["end"], "PRED") for span in cer_pipeline(text)
]

gt_annotations = [
    (row.start, row.end, "GT") for row in test_annotations_df.loc[note_id].itertuples()
]

show_span_line_markup(text, predicted_annotations + gt_annotations)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


# linking Model

In [ ]:
# SG = SnomedGraph.from_rf2("/content/Notes2SNOMED/data/rf2-d2-osx-x64/bin/rf2")
SG = SnomedGraph.from_rf2("SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z") # Unable to download SNOMED_CT RF2

In [ ]:
concepts_in_scope = (
    SG.get_descendants(71388002)
    | SG.get_descendants(123037004)
    | SG.get_descendants(404684003)
)

In [ ]:
concepts_in_scope = [
    SG.get_concept_details(a) for a in annotations_df.concept_id.unique()
]

print(f"{len(concepts_in_scope)} concepts have been selected.")


In [ ]:
kb_model = SentenceTransformer(kb_embedding_model_id)

kb_sft_examples = [
    InputExample(texts=[syn1, syn2], label=1)
    for concept in tqdm(concepts_in_scope)
    for syn1, syn2 in combinations(concept.synonyms, 2)
]

kb_sft_dataloader = DataLoader(kb_sft_examples, shuffle=True, batch_size=32)

kb_sft_loss = losses.ContrastiveLoss(kb_model)

kb_model.fit(
    train_objectives=[(kb_sft_dataloader, kb_sft_loss)],
    epochs=2,
    warmup_steps=100,
    checkpoint_path="~/temp/ke_encoder",
)

kb_model.save("kb_model")

In [ ]:
class Linker:
    def __init__(self, encoder, context_window_width=0):
        self.encoder = encoder
        self.entity_index = KeyedVectors(self.encoder[1].word_embedding_dimension)
        self.context_index = dict()
        self.history = dict()
        self.context_window_width = context_window_width

    def add_context(self, row):
        window_start = max(0, row.start - self.context_window_width)
        window_end = min(row.end + self.context_window_width, len(row.text))
        return row.text[window_start:window_end]

    def add_entity(self, row):
        return row.text[row.start : row.end]

    def fit(self, df=None, snomed_concepts=None):
        # Create a map from the entities to the concepts and contexts in which they appear
        if df is not None:
            for row in df.itertuples():
                entity = self.add_entity(row)
                context = self.add_context(row)
                map_ = self.history.get(entity, dict())
                contexts = map_.get(row.concept_id, list())
                contexts.append(context)
                map_[row.concept_id] = contexts
                self.history[entity] = map_

        # Add SNOMED CT codes for lookup
        if snomed_concepts is not None:
            for c in snomed_concepts:
                for syn in c.synonyms:
                    map_ = self.history.get(syn, dict())
                    contexts = map_.get(c.sctid, list())
                    contexts.append(syn)
                    map_[c.sctid] = contexts
                    self.history[syn] = map_

        # Create indexes to help disambiguate entities by their contexts
        for entity, map_ in tqdm(self.history.items()):
            keys = [
                (concept_id, occurance)
                for concept_id, contexts in map_.items()
                for occurance, context in enumerate(contexts)
            ]
            contexts = [context for contexts in map_.values() for context in contexts]
            vectors = self.encoder.encode(contexts)
            index = KeyedVectors(self.encoder[1].word_embedding_dimension)
            index.add_vectors(keys, vectors)
            self.context_index[entity] = index

        # Now create the top-level entity index
        keys = list(self.history.keys())
        vectors = self.encoder.encode(keys)
        self.entity_index.add_vectors(keys, vectors)

    def link(self, row):
        entity = self.add_entity(row)
        context = self.add_context(row)
        vec = self.encoder.encode(entity)
        nearest_entity = self.entity_index.most_similar(vec, topn=1)[0][0]
        index = self.context_index.get(nearest_entity, None)

        if index:
            vec = self.encoder.encode(context)
            key, score = index.most_similar(vec, topn=1)[0]
            sctid, _ = key
            return sctid
        else:
            return None


In [ ]:
linker_training_df = training_notes_df.join(training_annotations_df)
linker_test_df = test_notes_df.join(test_annotations_df)
def evaluate_linker(linker, df):
    n_correct = 0
    n_examples = df.shape[0]

    for _, row in tqdm(df.iterrows(), total=n_examples):
        sctid = linker.link(row)
        if row["concept_id"] == sctid:
            n_correct += 1

    return n_correct / n_examples


In [ ]:
for context_window_width in tqdm([5, 8, 10, 12]):
    linker = Linker(kb_model, context_window_width)
    linker.fit(linker_training_df, concepts_in_scope)
    acc = evaluate_linker(linker, linker_test_df)
    print(f"Context Window Width: {context_window_width}\tAccuracy: {acc}")


In [ ]:
linker = Linker(kb_model, 12)
linker.fit(linker_training_df, concepts_in_scope)

with open("linker.pickle", "wb") as f:
    pickle.dump(linker, f)

with open("linker.pickle", "rb") as f:
    linker = pickle.load(f)


In [ ]:
def predict(df):
    # One note at a time...
    for row in tqdm(df.itertuples(), total=df.shape[0]):
        # Tokenize the entire discharge note
        tokenized = cer_tokenizer(
            row.text,
            return_offsets_mapping=False,
            add_special_tokens=False,
            truncation=False,
        )

        global_offset = 0
        global_offset_mapping = []

        # Adjust the token offsets so that they match the original document
        for input_id in tokenized["input_ids"]:
            token = cer_tokenizer.decode(input_id)
            pos = row.text[global_offset:].find(token)
            start = global_offset + pos
            end = global_offset + pos + len(token)
            global_offset = end
            global_offset_mapping.append((start, end))

        chunk_start_idx = 0
        chunk_end_idx = 0

        # Process the document in chunks of 512 tokens chunk at a time
        for offset_chunk in chunked(global_offset_mapping, max_seq_len - 2):
            chunk_start_idx = chunk_end_idx
            chunk_end_idx = offset_chunk[-1][1]
            chunk_text = row.text[chunk_start_idx:chunk_end_idx]

            # Iterate through the detected entities and link them
            for entity in cer_pipeline(chunk_text):
                example = pd.Series(
                    {
                        # +1 to account for the [CLS] token
                        "start": entity["start"] + chunk_start_idx + 1,
                        "end": entity["end"] + chunk_start_idx,
                        "text": row.text,
                    }
                )
                sctid = linker.link(example)

                # Only yield matches where the Linker returned something
                if sctid:
                    yield {
                        "note_id": row.Index,
                        "start": example["start"],
                        "end": example["end"],
                        "concept_id": sctid,
                        # The following are useful for debugging and analysis
                        "FSN": SG.get_concept_details(sctid).fsn,
                        "entity": row.text[example["start"] : example["end"]],
                        "tokenizer_word": entity["word"],
                    }


preds_df = pd.DataFrame(list(predict(test_notes_df)))


In [ ]:
note_id = "10807423-DS-19"
text = test_notes_df.loc[note_id].text

predicted_annotations = [
    (row.start, row.end, f"P_{row.concept_id}")
    for row in preds_df.set_index("note_id").loc[note_id].itertuples()
]

gt_annotations = [
    (row.start, row.end, f"GT_{row.concept_id}")
    for row in test_annotations_df.loc[note_id].itertuples()
]

show_span_line_markup(text, predicted_annotations + gt_annotations)


In [ ]:
def iou_per_class(
    user_annotations: pd.DataFrame, target_annotations: pd.DataFrame
) -> List[float]:
    """
    Calculate the IoU metric for each class in a set of annotations.
    """
    # Get mapping from note_id to index in array
    docs = np.unique(
        np.concatenate([user_annotations.note_id, target_annotations.note_id])
    )
    doc_index_mapping = dict(zip(docs, range(len(docs))))

    # Identify union of categories in GT and PRED
    cats = np.unique(
        np.concatenate([user_annotations.concept_id, target_annotations.concept_id])
    )

    # Find max character index in GT or PRED
    max_end = np.max(np.concatenate([user_annotations.end, target_annotations.end]))

    # Populate matrices for keeping track of character class categorization
    def populate_char_mtx(n_rows, n_cols, annot_df):
        mtx = sp.lil_array((n_rows, n_cols), dtype=np.uint64)
        for row in annot_df.itertuples():
            doc_index = doc_index_mapping[row.note_id]
            mtx[doc_index, row.start : row.end] = row.concept_id  # noqa: E203
        return mtx.tocsr()

    gt_mtx = populate_char_mtx(docs.shape[0], max_end, target_annotations)
    pred_mtx = populate_char_mtx(docs.shape[0], max_end, user_annotations)

    # Calculate IoU per category
    ious = []
    for cat in cats:
        gt_cat = gt_mtx == cat
        pred_cat = pred_mtx == cat
        # sparse matrices don't support bitwise operators, but the _cat matrices
        # have bool dtypes so when we multiply/add them we end up with only T/F values
        intersection = gt_cat * pred_cat
        union = gt_cat + pred_cat
        iou = intersection.sum() / union.sum()
        ious.append(iou)

    return ious

### Save final CER model

In [ ]:
training_args.num_train_epochs = 1

trainer = Trainer(
    model=cer_model,
    args=training_args,
    train_dataset=test,
    eval_dataset=test,
    tokenizer=cer_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("cer_model")

### Save linker


In [ ]:
kb_model = SentenceTransformer("kb_model")
linker = Linker(kb_model, 12)
linker.fit(notes_df.join(annotations_df), concepts_in_scope)

with open("linker.pickle", "wb") as f:
    pickle.dump(linker, f)

### Submission Code

In [ ]:
from pathlib import Path

import dill as pickle
import pandas as pd
from loguru import logger
from more_itertools import chunked
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer, DebertaV2ForTokenClassification, pipeline

NOTES_PATH = Path("data/test_notes.csv")
SUBMISSION_PATH = Path("submission.csv")
LINKER_PATH = Path("linker.pickle")
CER_MODEL_PATH = Path("cer_model")

CONTEXT_WINDOW_WIDTH = 12
MAX_SEQ_LEN = 512
USE_LORA = False


def load_cer_pipeline():
    label2id = {"O": 0, "B-clinical_entity": 1, "I-clinical_entity": 2}

    id2label = {v: k for k, v in label2id.items()}

    cer_tokenizer = AutoTokenizer.from_pretrained(
        CER_MODEL_PATH, model_max_length=MAX_SEQ_LEN
    )

    if USE_LORA:
        config = PeftConfig.from_pretrained(CER_MODEL_PATH)

        cer_model = DebertaV2ForTokenClassification.from_pretrained(
            pretrained_model_name_or_path=config.base_model_name_or_path,
            num_labels=3,
            id2label=id2label,
            label2id=label2id,
        )
        cer_model = PeftModel.from_pretrained(cer_model, CER_MODEL_PATH)
    else:
        cer_model = DebertaV2ForTokenClassification.from_pretrained(
            pretrained_model_name_or_path=CER_MODEL_PATH,
            num_labels=3,
            id2label=id2label,
            label2id=label2id,
        )

    cer_pipeline = pipeline(
        task="token-classification",
        model=cer_model,
        tokenizer=cer_tokenizer,
        aggregation_strategy="first",
        device="cpu",
    )
    return cer_pipeline


def main():
    # columns are note_id, text
    logger.info("Reading in notes data.")
    notes = pd.read_csv(NOTES_PATH)
    logger.info(f"Found {notes.shape[0]} notes.")
    spans = []

    # Load model components
    logger.info("Loading CER pipeline.")
    cer_pipeline = load_cer_pipeline()
    cer_tokenizer = cer_pipeline.tokenizer

    logger.info("Loading Linker")
    with open(LINKER_PATH, "rb") as f:
        linker = pickle.load(f)

    # Process one note at a time...
    logger.info("Processing notes.")
    for row in notes.itertuples():
        # Tokenize the entire discharge note
        tokenized = cer_tokenizer(
            row.text,
            return_offsets_mapping=False,
            add_special_tokens=False,
            truncation=False,
        )

        global_offset = 0
        global_offset_mapping = []

        # Adjust the token offsets so that they match the original document
        for input_id in tokenized["input_ids"]:
            token = cer_tokenizer.decode(input_id)
            pos = row.text[global_offset:].find(token)
            start = global_offset + pos
            end = global_offset + pos + len(token)
            global_offset = end
            global_offset_mapping.append((start, end))

        chunk_start_idx = 0
        chunk_end_idx = 0

        # Process the document in chunks of 512 tokens chunk at a time
        for offset_chunk in chunked(global_offset_mapping, MAX_SEQ_LEN - 2):
            chunk_start_idx = chunk_end_idx
            chunk_end_idx = offset_chunk[-1][1]
            chunk_text = row.text[chunk_start_idx:chunk_end_idx]

            # ...one matched clinical entity at a time
            # Iterate through the detected entities and link them
            for entity in cer_pipeline(chunk_text):
                example = pd.Series(
                    {
                        # +1 to account for the [CLS] token
                        "start": entity["start"] + chunk_start_idx + 1,
                        "end": entity["end"] + chunk_start_idx,
                        "text": row.text,
                    }
                )
                sctid = linker.link(example)
                if sctid:
                    spans.append(
                        {
                            "note_id": row.Index,
                            "start": example["start"],
                            "end": example["end"],
                            "concept_id": sctid,
                        }
                    )

    logger.info(f"Generated {len(spans)} annotated spans.")
    spans_df = pd.DataFrame(spans)
    spans_df.to_csv(SUBMISSION_PATH, index=False)
    logger.info("Finished.")


if __name__ == "__main__":
    main()

# Train Tokenizer

#### MedCAT BPE

In [ ]:
# Install medcat
! pip install medcat~=1.10.0
# try:
#     from medcat.cat import CAT
# except:
#     print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
#     exit()

In [ ]:
import gensim
import pandas as pd
import numpy as np
from tokenizers import ByteLevelBPETokenizer
from gensim.models import Word2Vec

In [ ]:
DATA_DIR = "./data_p4.1/"
! DATA_DIR="./data_p4.1/"

In [ ]:
!mkdir ./models
!wget -N https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/noteevents.csv -P $DATA_DIR

In [ ]:
# To train the tokenizer we will use all the data we have from our dummy dataset.
df = pd.read_csv(DATA_DIR + "noteevents.csv")
df.head()
# notes_df.head()
# df = notes_df

In [ ]:
# The tokenizers from huggingface require us to save all the text used for
#training into one/multiple text files.
f = open(DATA_DIR + "tok_data.txt", 'w')
for text in df['text'].values:
    #We'll remove new lines, so that we have one document in one line
    text = text.strip().replace("\n", ' ')
    f.write(text.lower()) # Lowercase text to remove noise
    f.write("\n")
f.close()

In [ ]:
# Create, train and save the tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(DATA_DIR + "tok_data.txt")
tokenizer.save("./models/bbpe")

In [ ]:
# Now we tokenize all the text we have and train word2vec
f = open(DATA_DIR + "tok_data.txt", 'r')
# Note that if you have a very large dataset, use iterators that
#read the text line by line from the file, do not load the whole file
#into memory.
data = []
for line in f:
    data.append(tokenizer.encode(line).tokens)
w2v = Word2Vec(data, vector_size=300, min_count=1)

In [ ]:
# Check is word2vec trained, Ġ - for this tokenizer denotes start of word (a space)
w2v.wv.most_similar('Ġcancer')

In [ ]:
# Now we just have to create the embeddings matrix
embeddings = []
for i in range(tokenizer.get_vocab_size()):
    word = tokenizer.id_to_token(i)
    if word in w2v.wv:
        embeddings.append(w2v.wv[word])
    else:
        # Assign a random vector if the word was not frequent enough to receive
        #an embedding
        embeddings.append(np.random.rand(300))

In [ ]:
# Save the embeddings
np.save(open("./models/medCAT_embeddings.npy", 'wb'), np.array(embeddings))

### MinBPE

https://github.com/karpathy/minbpe/tree/master

In [ ]:

!git clone https://github.com/karpathy/minbpe.git


In [ ]:
### B

from minbpe.minbpe.basic import BasicTokenizer
basic_tokenizer = BasicTokenizer()
text = "aaabdaaabac"
basic_tokenizer.train(text, 256 + 3) # 256 are the byte tokens, then do 3 merges
print(basic_tokenizer.encode(text))
# [258, 100, 258, 97, 99]
print(tokenizer.decode([258, 100, 258, 97, 99]))
# aaabdaaabac
basic_tokenizer.save("toy")
# writes two files: toy.model (for loading) and toy.vocab (for viewing)

text = "hello123!!!? (안녕하세요!) 😉"

# tiktoken
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode(text))
# [15339, 4513, 12340, 30, 320, 31495, 230, 75265, 243, 92245, 16715, 57037]

# gpt4tokenizer
from minbpe.minbpe.gpt4 import GPT4Tokenizer
gpt4_tokenizer = GPT4Tokenizer()
print(gpt4_tokenizer.encode(text))

In [ ]:
# If using the adaptor, ignore the warning:
# "The model 'PeftModelForTokenClassification' is not supported for token-classification."
# The PEFT model is wrapped just fine and will work within the pipeline.
# N.B. moving model to CPU makes inference slower, but enables us to feed the pipeline
# directly with strings.
cer_pipeline = pipeline(
    task="token-classification",
    model=cer_model,
    tokenizer= gpt4_tokenizer,
    aggregation_strategy="first",
    device="cuda", # cpu
)


# Docker

In [ ]:
%%shell

sudo apt update -qq

sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

sudo apt update -qq

sudo apt install docker-ce

docker

# Prep Submission

In [ ]:
!git clone https://github.com/drivendataorg/snomed-ct-entity-linking-runtime.git

Cloning into 'snomed-ct-entity-linking-runtime'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 103 (delta 46), reused 84 (delta 35), pack-reused 0
Receiving objects: 100% (103/103), 107.28 KiB | 17.88 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
make pull
make pack-example
make test-submission

#Lancedb

In [ ]:
ls

In [ ]:
!pip install lancedb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('FremyCompany/BioLORD-2023')

In [ ]:
# import Notes2SNOMED.lancedb as lancedb
import lancedb
db = lancedb.connect('./drive/MyDrive/collab_mount/lancedb')


tbl = db.open_table("definitions")

search_vec = model.encode("red tonsils inflammation with pus")
tbl.search(search_vec).limit(5).to_pandas()